In [1]:
import pandas as pd
df = pd.read_csv("/Users/kartikayluthra/Desktop/finance-qa-/data/processed/train_sample.csv")
sample_df = df.sample(1000, random_state=42)
sample_df = sample_df[['user', 'assistant']].dropna()
print(f"Loaded {len(sample_df)} samples")

Loaded 1000 samples


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"


In [3]:
from datasets import Dataset

dataset = Dataset.from_pandas(sample_df)
dataset = dataset.train_test_split(test_size = 0.1, seed = 42)
train_dataset = dataset['train']
test_dataset = dataset['test']
eval_dataset = dataset['test']


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "google/t5-efficient-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [5]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # attention query/value projections
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

In [6]:
max_input_length = 32
max_target_length = 32

def tokenize_function(example):
    model_inputs = tokenizer(example['user'], max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['assistant'], max_length=max_target_length, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
import transformers
print(transformers.__version__)


4.56.1


In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/Users/kartikayluthra/Desktop/finance-qa-/models/t5-small-finance",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    fp16=False,
    predict_with_generate=True,
    save_total_limit=2,
    logging_strategy = "no",
    save_strategy="epoch"
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [20]:
import torch

device = torch.device("cpu")
model.to(device)


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 256)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 256)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=256, out_features=256, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=256, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=256, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [21]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset = tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator
)


/var/folders/1z/npbdg1px16jdq9vjn93zzfcm0000gn/T/ipykernel_20154/262139089.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [22]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=1800, training_loss=7.392017144097222, metrics={'train_runtime': 151.447, 'train_samples_per_second': 11.885, 'train_steps_per_second': 11.885, 'total_flos': 2009514049536.0, 'train_loss': 7.392017144097222, 'epoch': 2.0})